In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from glob import glob
import os
import TOC_functions
%matplotlib widget 

In [ ]:
root_folder = os.path.expanduser('C:/Users/ica/OneDrive - Plymouth Marine Laboratory/vscode/TOC_analysis/data/11')  # yourpath
picarro_df = TOC_functions.merge_picarro_files(root_folder)
picarro_df.set_index('datetime', inplace=True)

In [ ]:
# filter by time

start_time = datetime.datetime(2024, 11, 20, 11, 0, 0)
end_time = datetime.datetime(2024, 11, 20, 16, 0, 0)

In [ ]:
df = picarro_df.loc[start_time:end_time]
df = TOC_functions.qc(df)

In [ ]:
TOC_df = TOC_functions.TOC_df(df)


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(10, 8))

# Plot for CO
ax1.plot(df.index, df.CO, label='CO / ppm', color='blue')
ax1.set_ylabel('CO / ppm', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot for CH4
ax2.plot(df.index, df.CH4_corrected, label='CH4 / ppm', color='red')
ax2.set_ylabel('CH4 / ppm', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Plot for CO2
ax3.plot(df.index, df.CO2_corrected, label='CO2 / ppm', color='green')
ax3.set_xlabel('Date')
ax3.set_ylabel('CO2 / ppm', color='green')
ax3.tick_params(axis='y', labelcolor='green')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
def calculate_ox(TOC_df):
    ch4_ox = []

    for i in range(0, len(TOC_df)-1, 2):
        pair = TOC_df.iloc[i:i+2]

        ch4_catalyst = pair['avg_ch4_catalyst'].sum()
        ch4_ambient = pair['avg_ch4_ambient'].sum()
        # Calculate CH4 oxidation
        ch4_ox_value = 1 - (ch4_catalyst / ch4_ambient)
        
        # Use the mean timestamp of the pair
        ch4_ox.append((pair.index.mean(), ch4_ox_value))
    return pd.DataFrame(ch4_ox, columns=['datetime', 'ch4_ox']).set_index('datetime')
    
   

In [ ]:
ch4_ox_df = calculate_ox(TOC_df)
df_resampled = df.resample('1T').mean()  # Resample to 1-minute intervals
ch4_ox_df_resampled = ch4_ox_df.resample('1T').mean()
ch4_ox_df_resampled['H2O'] = df_resampled['H2O']



In [ ]:
plt.figure()
plt.scatter(ch4_ox_df_resampled.H2O, ch4_ox_df_resampled.ch4_ox, label='CH4 Oxidation efficiency', color = 'blue')
plt.xlabel('% H2O')  # Replace with your actual label for the x-axis
plt.ylabel('CH4 Oxidation Efficiency')  # Label for the y-axis
plt.show()
